In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
mydf = spark.read.parquet("s3://zandras3/502final/df.parquet/*")

In [5]:
mydf.printSchema()

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: stri

In [6]:
mydf.show(5)

+-------------+--------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+---------+--------------------+
|GLOBALEVENTID| SQLDATE|MonthYear|Year|FractionDate|Actor1Code|Actor1Name|Acto

In [6]:
df_feat = mydf.select(mydf.SQLDATE, mydf.MonthYear, mydf.Year, 
                      mydf.Actor1Code, mydf.Actor1Name,mydf.Actor1CountryCode,
                      mydf.Actor2Code, mydf.Actor2Name, mydf.Actor2CountryCode, 
                      mydf.QuadClass, mydf.GoldsteinScale, mydf.NumSources, 
                      mydf.AvgTone, mydf.Actor1Geo_Type, mydf.Actor1Geo_CountryCode,
                     mydf.Actor1Geo_Lat, mydf.Actor1Geo_Long, mydf.Actor2Geo_Type, 
                      mydf.Actor2Geo_CountryCode, mydf.Actor2Geo_Lat, mydf.Actor2Geo_Long,
                     mydf.ActionGeo_Type, mydf.ActionGeo_CountryCode,
                     mydf.ActionGeo_Lat, mydf.ActionGeo_Long, mydf.SOURCEURL)

In [7]:
df_feat.printSchema()

root
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- QuadClass: string (nullable = true)
 |-- GoldsteinScale: string (nullable = true)
 |-- NumSources: string (nullable = true)
 |-- AvgTone: string (nullable = true)
 |-- Actor1Geo_Type: string (nullable = true)
 |-- Actor1Geo_CountryCode: string (nullable = true)
 |-- Actor1Geo_Lat: string (nullable = true)
 |-- Actor1Geo_Long: string (nullable = true)
 |-- Actor2Geo_Type: string (nullable = true)
 |-- Actor2Geo_CountryCode: string (nullable = true)
 |-- Actor2Geo_Lat: string (nullable = true)
 |-- Actor2Geo_Long: string (nullable = true)
 |-- ActionGeo_Type: string (nullable = true)
 |-- ActionGeo_Coun

In [9]:
df_feat.show(15)

+--------+---------+----+----------+------------+-----------------+---------+--------------+----------+-----------------+--------------+---------------------+-------------+--------------+--------------+---------------------+-------------+--------------+--------------+---------------------+-------------+--------------+
| SQLDATE|MonthYear|Year|Actor2Code|  Actor2Name|Actor2CountryCode|QuadClass|GoldsteinScale|NumSources|          AvgTone|Actor1Geo_Type|Actor1Geo_CountryCode|Actor1Geo_Lat|Actor1Geo_Long|Actor2Geo_Type|Actor2Geo_CountryCode|Actor2Geo_Lat|Actor2Geo_Long|ActionGeo_Type|ActionGeo_CountryCode|ActionGeo_Lat|ActionGeo_Long|
+--------+---------+----+----------+------------+-----------------+---------+--------------+----------+-----------------+--------------+---------------------+-------------+--------------+--------------+---------------------+-------------+--------------+--------------+---------------------+-------------+--------------+
|20151224|   201512|2015|       BUS|    

### Sentiment Analysis

In [8]:
from pyspark.sql.functions import avg

In [13]:
tone = df_feat.select("Year","AvgTone").groupBy("Year").agg(avg("AvgTone"))

In [14]:
tone.head(10)

[Row(Year='2016', avg(AvgTone)=-2.092142628981268),
 Row(Year='2006', avg(AvgTone)=-1.5654240261469206),
 Row(Year='2007', avg(AvgTone)=-1.4407238930749484),
 Row(Year='2015', avg(AvgTone)=-2.121795877376201)]

In [23]:
tones_pd = tone.toPandas()

Average tone analysis for each year in the dataset:

In [24]:
tones_pd

Year  avg(AvgTone)
0  2016     -2.092143
1  2006     -1.565424
2  2007     -1.440724
3  2015     -2.121796

In [16]:
# df_feat = df_feat.withColumn("AvgTone", df_feat["AvgTone"].cast("double"))

Use Dask to visualize and filter data

In [9]:
import dask
import dask.dataframe as dd
import pandas as pd

In [10]:
df_feat.write.save("s3://zandras3/502final/feat2_news.csv", format='csv', header=True)

In [11]:
dask_feat = dd.read_csv('s3://zandras3/502final/feat2_news.csv/*.csv', 
                        dtype={'ActionGeo_Long': 'object',
                       'Actor1Geo_Long': 'object',
                       'Actor2Geo_Long': 'object'})

In [12]:
dask_feat

Dask DataFrame Structure:
                SQLDATE MonthYear   Year Actor1Code Actor1Name Actor1CountryCode Actor2Code Actor2Name Actor2CountryCode QuadClass GoldsteinScale NumSources  AvgTone Actor1Geo_Type Actor1Geo_CountryCode Actor1Geo_Lat Actor1Geo_Long Actor2Geo_Type Actor2Geo_CountryCode Actor2Geo_Lat Actor2Geo_Long ActionGeo_Type ActionGeo_CountryCode ActionGeo_Lat ActionGeo_Long SOURCEURL
npartitions=297                                                                                                                                                                                                                                                                                                                                                                      
                  int64     int64  int64     object     object            object     object     object            object     int64        float64      int64  float64          int64                object       float64         object          int64                object       float64         object          int64                object       float64         object    object
                    ...       ...    ...        ...        ...               ...        ...        ...               ...       ...            ...        ...      ...            ...                   ...           ...            ...            ...                   ...           ...            ...            ...                   ...           ...            ...       ...
...                 ...       ...    ...        ...        ...               ...        ...        ...               ...       ...            ...        ...      ...            ...                   ...           ...            ...            ...                   ...           ...            ...            ...                   ...           ...            ...       ...
                    ...       ...    ...        ...        ...               ...        ...        ...               ...       ...            ...        ...      ...            ...                   ...           ...            ...            ...                   ...           ...            ...            ...                   ...           ...            ...       ...
                    ...       ...    ...        ...        ...               ...        ...        ...               ...       ...            ...        ...      ...            ...                   ...           ...            ...            ...                   ...           ...            ...            ...                   ...           ...            ...       ...
Dask Name: from-delayed, 891 tasks

Overview of news that have the most strong positive tone

In [13]:
most_extreme_pos_tone = dask_feat[['MonthYear', 'AvgTone', 'Actor1Name', 'Actor2Name', 'SOURCEURL']].nlargest(10, "AvgTone").compute()

In [18]:
pd.set_option('display.max_colwidth', -1)
most_extreme_pos_tone

/home/hadoop/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


MonthYear    AvgTone    Actor1Name      Actor2Name  \
34409   201603     29.881154  CANADA        UNITED STATES    
34410   201603     29.844291  CANADA        UNITED STATES    
167856  201608     26.229508  STUDENT       UNIVERSITY       
269452  201611     25.431034  BUSINESS      NaN              
269478  201611     25.431034  BUSINESS      UNITED KINGDOM   
192905  201607     25.000000  CITIZEN       MEDIA            
236906  201606     24.590164  NEWS OUTLET   ADVOCATE         
123131  201603     24.444444  CREDIT UNION  NaN              
123133  201603     24.444444  CREDIT UNION  NaN              
123195  201603     24.444444  CREDIT UNION  CREDIT UNION     

                                                                                                                                                 SOURCEURL  
34409   https://en.wikinews.org/wiki/Canada=sage=50=pro=advisor=support=number=1~844~313~4859=sage=pro=tech=support=number=Sage=50=accounts=support=number  
34410   https://en.wikinews.org/wiki/Dial=sage=50=canada=support=number=1~844~313~4859=sage=instant=payroll=helpline=SAGE=50=premium=support=number         
167856  http://readme.readmedia.com/Frostburg-Showcases-Outstanding-Student-Art-and-Creativity-in-Communications-at-Frosty-Awards/merit-61554               
269452  http://www.bristol247.com/channel/food-drink/news/awards/bristol-good-food-awards-2016-winners-reveal                                               
269478  http://www.bristol247.com/channel/food-drink/news/awards/bristol-good-food-awards-2016-winners-reveal                                               
192905  https://www.yahoo.com/news/media-receive-brotherly-love-democratic-042741452.html                                                                   
236906  http://www.playbill.com/article/photos-harvey-fierstein-honored-as-lgbt-trailblazer                                                                 
123131  https://www.cuinsight.com/wegner-winners-share-leadership-advice.html                                                                               
123133  https://www.cuinsight.com/wegner-winners-share-leadership-advice.html                                                                               
123195  https://www.cuinsight.com/wegner-winners-share-leadership-advice.html

Overview of news that have the most strong negative tone

In [19]:
most_extreme_tone_neg = dask_feat[['MonthYear', 'AvgTone', 'Actor1Name', 'Actor2Name', 'SOURCEURL']].nsmallest(10, "AvgTone").compute()

In [20]:
most_extreme_tone_neg

MonthYear    AvgTone      Actor1Name      Actor2Name  \
101147  201603    -77.878034  THE US          UNITED KINGDOM   
1463    201602    -59.471684  SOUTH CAROLINA  NaN              
1494    201602    -59.471684  NEW YORK        NaN              
58732   201603    -57.097065  UNITED KINGDOM  NaN              
155764  201607    -50.000000  SUDAN           FRANCE           
35882   201608    -40.000000  PRISON          ILLINOIS         
68577   201609    -39.215686  PROSECUTOR      IOWA             
269091  201606    -39.215686  EMPLOYEE        CHICAGO          
269663  201606    -39.215686  CHICAGO         EMPLOYEE         
108904  201608    -38.461538  NaN             SAO PAULO        

                                                                                                               SOURCEURL  
101147  http://www.wandsworthguardian.co.uk/news/14340813.display/                                                        
1463    http://freerepublic.com/focus/f-news/3398001/posts                                                                
1494    http://freerepublic.com/focus/f-news/3398001/posts                                                                
58732   http://www.dailymail.co.uk/news/article-3489982/Muslim-extremist-jailed-firebombing-sells-organic-beard-oil.html  
155764  Multiple Sources/BBC Monitoring/(c) BBC                                                                           
35882   https://www.yahoo.com/news/mother-cancer-convicted-killing-disabled-172017802.html                                
68577   https://www.yahoo.com/news/prosecutor-resigns-arrest-allegedly-stealing-163018607.html                            
269091  https://www.yahoo.com/news/juvenile-suspect-arrested-fatal-shooting-225010116.html                                
269663  https://www.yahoo.com/news/juvenile-suspect-arrested-fatal-shooting-225010116.html                                
108904  https://www.yahoo.com/news/tear-gas-fired-impeachment-protesters-063057459.html

### News Related to Certain Countries

Each country's news and their mean values for the average tones:

In [23]:
country_mean = dask_feat.groupby('Actor2CountryCode').AvgTone.mean().compute()

In [26]:
country_mean

Actor2CountryCode
ABW    0.410868
AFG   -3.576521
AFR   -1.495684
AGO   -1.553089
AIA    0.860718
         ...   
WLF    0.373889
SCN    1.604239
CAU    1.999307
NAF   -4.998527
PRI   -2.413313
Name: AvgTone, Length: 222, dtype: float64

In [29]:
country_mean_pd = pd.DataFrame({'Country':country_mean.index, 'AvgTone_mean':country_mean.values})

In [30]:
country_mean_pd 

Country  AvgTone_mean
0    ABW     0.410868    
1    AFG    -3.576521    
2    AFR    -1.495684    
3    AGO    -1.553089    
4    AIA     0.860718    
..   ...          ...    
217  WLF     0.373889    
218  SCN     1.604239    
219  CAU     1.999307    
220  NAF    -4.998527    
221  PRI    -2.413313    

[222 rows x 2 columns]

In [35]:
pd.set_option('display.max_rows', 250)
country_mean_pd.sort_values('AvgTone_mean')

Country  AvgTone_mean
220  NAF    -4.998527    
208  YEM    -4.256367    
182  SYR    -4.253860    
144  NRU    -4.218909    
173  SOM    -4.140004    
14   BDI    -4.126763    
158  PSE    -4.121354    
39   COD    -3.986905    
31   CAF    -3.901824    
191  TUN    -3.659807    
90   IRQ    -3.639573    
17   BFA    -3.628236    
137  NER    -3.612901    
192  TUR    -3.602452    
1    AFG    -3.576521    
127  MLI    -3.565143    
122  MDV    -3.554458    
40   COG    -3.550657    
60   ERI    -3.515343    
15   BEL    -3.494339    
109  LBY    -3.447016    
18   BGD    -3.324835    
172  SLV    -3.319493    
175  SSD    -3.284639    
183  TCD    -3.276302    
126  MKD    -3.270410    
165  SDN    -3.178934    
216  ROM    -3.169027    
147  PAK    -3.139422    
92   ISR    -3.116771    
82   HND    -3.084264    
38   CMR    -3.054336    
107  LBN    -3.009297    
164  SAU    -2.964689    
155  PRK    -2.958683    
196  UKR    -2.927530    
37   CIV    -2.901354    
211  ZWE    -2.808536    
77   GRC    -2.754916    
160  RUS    -2.724904    
59   EGY    -2.654153    
8    ARM    -2.589226    
202  VEN    -2.586906    
195  UGA    -2.556821    
79   GTM    -2.528644    
84   HTI    -2.522542    
89   IRN    -2.519419    
69   GAB    -2.478356    
95   JOR    -2.416734    
98   KEN    -2.415476    
221  PRI    -2.413313    
20   BHR    -2.381150    
51   DEU    -2.368369    
138  NGA    -2.350587    
132  MRT    -2.349056    
26   BRA    -2.306315    
63   ETH    -2.292431    
148  PAN    -2.234982    
87   IND    -2.223453    
52   DJI    -2.184357    
67   FRA    -2.172252    
134  MWI    -2.154804    
163  SAS    -2.148237    
86   IDN    -2.145735    
58   ECU    -2.136548    
5    ALB    -2.133496    
123  MEA    -2.066737    
56   DZA    -2.049862    
71   GEO    -2.043236    
124  MEX    -2.034500    
12   AUT    -2.023333    
151  PHL    -2.019105    
150  PGS    -1.957749    
161  RWA    -1.945812    
198  USA    -1.933952    
83   HRV    -1.913650    
174  SRB    -1.876262    
207  WST    -1.864943    
13   AZE    -1.863106    
179  SWE    -1.837476    
129  MMR    -1.799013    
154  POL    -1.785910    
135  MYS    -1.765589    
209  ZAF    -1.764648    
153  PNG    -1.748004    
64   EUR    -1.744984    
74   GMB    -1.741682    
140  NLD    -1.668286    
21   BHS    -1.652301    
80   GUY    -1.625513    
103  KOR    -1.621296    
185  THA    -1.618371    
105  LAM    -1.616492    
167  SEN    -1.611433    
93   ITA    -1.606446    
100  KHM    -1.601317    
23   BLZ    -1.560444    
34   CHE    -1.557812    
3    AGO    -1.553089    
11   AUS    -1.521456    
42   COL    -1.507639    
85   HUN    -1.495746    
2    AFR    -1.495684    
131  MOZ    -1.491532    
190  TTO    -1.475124    
197  URY    -1.455612    
166  SEA    -1.444724    
25   BOL    -1.431414    
73   GIN    -1.424268    
157  PRY    -1.421062    
186  TJK    -1.419946    
70   GBR    -1.416844    
118  MAR    -1.395631    
136  NAM    -1.394754    
215  SAM    -1.369702    
104  KWT    -1.367788    
149  PER    -1.365045    
61   ESP    -1.348343    
55   DOM    -1.326969    
139  NIC    -1.309455    
49   CYP    -1.294533    
193  TWN    -1.279209    
142  NOR    -1.271829    
177  SUR    -1.226098    
113  LSO    -1.223898    
19   BGR    -1.223321    
119  MCO    -1.185971    
7    ARG    -1.177243    
199  UZB    -1.171317    
178  SVK    -1.160560    
88   IRL    -1.150464    
194  TZA    -1.138847    
143  NPL    -1.135126    
28   BRN    -1.118975    
16   BEN    -1.116616    
94   JAM    -1.115171    
50   CZE    -1.105476    
115  LUX    -1.082603    
112  LKA    -1.078423    
159  QAT    -1.075782    
32   CAN    -1.072715    
76   GNQ    -1.053928    
125  MHL    -1.007094    
200  VAT    -1.001262    
48   CYM    -0.981017    
96   JPN    -0.971672    
54   DNK    -0.970842    
171  SLE    -0.962289    
36   CHN    -0.938071    
114  LTU    -0.936896    
210  ZMB    -0.906700    
128  MLT    -0.897508    
156  PRT    -0.890368    


In [52]:
dask_feat.groupby('Actor2CountryCode').AvgTone.mean().compute()

Actor1CountryCode
ABW    0.485961
AFG   -3.680197
AFR   -1.514082
AGO   -1.417258
AIA    0.917811
         ...   
SCN   -0.194933
ROM   -3.137477
CAU   -1.220209
NAF   -4.362679
PRI   -1.167317
Name: AvgTone, Length: 222, dtype: float64

In [ ]:
dask_feat.groupby('Actor2Name').AvgTone.mean().compute()

News Related to China (Actor1 is China): 

In [51]:
dask_feat[dask_feat.Actor1CountryCode == 'CHN'].head(npartitions=-1)

SQLDATE  MonthYear  Year Actor1Code Actor1Name Actor1CountryCode  \
1073  20161223     201612  2016        CHN      CHINA               CHN   
1074  20161223     201612  2016        CHN      CHINA               CHN   
1075  20161223     201612  2016        CHN      CHINA               CHN   
1076  20161223     201612  2016        CHN      CHINA               CHN   
1077  20161223     201612  2016        CHN    CHINESE               CHN   

     Actor2Code Actor2Name Actor2CountryCode  QuadClass  ...  Actor1Geo_Lat  \
1073        NaN        NaN               NaN          1  ...        39.9289   
1074        NaN        NaN               NaN          1  ...        32.9889   
1075        NaN        NaN               NaN          1  ...        25.5775   
1076        NaN        NaN               NaN          1  ...        35.0000   
1077        NaN        NaN               NaN          1  ...        54.0000   

      Actor1Geo_Long  Actor2Geo_Type  Actor2Geo_CountryCode Actor2Geo_Lat  \
1073         116.388               0                    NaN           NaN   
1074         117.342               0                    NaN           NaN   
1075         115.256               0                    NaN           NaN   
1076             105               0                    NaN           NaN   
1077              -2               0                    NaN           NaN   

      Actor2Geo_Long ActionGeo_Type  ActionGeo_CountryCode ActionGeo_Lat  \
1073             NaN              4                     CH       39.9289   
1074             NaN              4                     CH       32.9889   
1075             NaN              4                     CH       25.5775   
1076             NaN              1                     CH       35.0000   
1077             NaN              1                     UK       54.0000   

      ActionGeo_Long  
1073         116.388  
1074         117.342  
1075         115.256  
1076             105  
1077              -2  

[5 rows x 25 columns]

Mean of average tones for News Related to China (Actor1 is China):

In [65]:
dask_feat[dask_feat.Actor1CountryCode == 'CHN'].AvgTone.mean().compute()

-0.961569938380273

Mean of average tones for News Related to China (Actor2 is China):

In [69]:
dask_feat[dask_feat.Actor2CountryCode == 'CHN'].AvgTone.mean().compute()

-0.938070685218778

Mean of average tones for News Related to US-China (Actor1 is China, Actor2 is the US):

In [37]:
china_news1 = dask_feat[dask_feat.Actor1CountryCode == 'CHN']
china_news1[china_news1.Actor2CountryCode == 'USA'].AvgTone.mean().compute()

-1.3531860494884245

Mean of average tones for News Related to US-China (Actor1 is the US, Actor2 is the China):

In [45]:
china_news2 = dask_feat[dask_feat.Actor2CountryCode == 'CHN']
china_news2[china_news2.Actor1CountryCode == 'USA'].AvgTone.mean().compute()

-1.3268823749714935

Mean of average tones for News Related to US-Russia (Actor1 is Russia, Actor2 is the US):

In [46]:
rus_news1 = dask_feat[dask_feat.Actor1CountryCode == 'RUS']
rus_news1[rus_news1.Actor2CountryCode == 'USA'].AvgTone.mean().compute()

-2.8030116784788057

Mean of average tones for News Related to US-Russia (Actor1 is the US, Actor2 is the Russia):

In [47]:
rus_news2 = dask_feat[dask_feat.Actor2CountryCode == 'RUS']
rus_news2[rus_news2.Actor1CountryCode == 'USA'].AvgTone.mean().compute()

-2.9090902048613976

This shows that Mean of average tones for News Related to the same country can be different, depends on if the country is Actor1 or Actor2.

News Related to China (Actor1 is China) that has most strong negative tones:

In [71]:
dask_feat[dask_feat.Actor1CountryCode == 'CHN'].nsmallest(10, "AvgTone").head()

SQLDATE  MonthYear  Year Actor1Code Actor1Name Actor1CountryCode  \
109570  20160728     201607  2016        CHN      CHINA               CHN   
290941  20160303     201603  2016        CHN     CANTON               CHN   
290942  20160303     201603  2016        CHN     CANTON               CHN   
186563  20160622     201606  2016        CHN      CHINA               CHN   
186564  20160622     201606  2016        CHN      CHINA               CHN   

       Actor2Code     Actor2Name Actor2CountryCode  QuadClass  ...  \
109570        USA       LAWRENCE               USA          4  ...   
290941        USA           OHIO               USA          3  ...   
290942        USA  UNITED STATES               USA          3  ...   
186563        NaN            NaN               NaN          4  ...   
186564        NaN            NaN               NaN          4  ...   

        Actor1Geo_Lat  Actor1Geo_Long  Actor2Geo_Type  Actor2Geo_CountryCode  \
109570        35.0000             105               1                     CH   
290941        40.8167        -81.3665               3                     US   
290942        40.8167        -81.3665               2                     US   
186563        35.0000             105               0                    NaN   
186564        35.0000             105               0                    NaN   

       Actor2Geo_Lat  Actor2Geo_Long ActionGeo_Type  ActionGeo_CountryCode  \
109570       35.0000             105              1                     CH   
290941       40.8167        -81.3665              3                     US   
290942       40.3736        -82.7755              2                     US   
186563           NaN             NaN              1                     CH   
186564           NaN             NaN              1                     CH   

       ActionGeo_Lat  ActionGeo_Long  
109570       35.0000             105  
290941       40.8167        -81.3665  
290942       40.3736        -82.7755  
186563       35.0000             105  
186564       35.0000             105  

[5 rows x 25 columns]

News Related to China (Actor1 is China) that has most strong positive tones:

In [75]:
dask_feat[dask_feat.Actor1CountryCode == 'CHN'].nlargest(10, "AvgTone").head()

SQLDATE  MonthYear  Year Actor1Code Actor1Name Actor1CountryCode  \
41394   20160523     201605  2016        CHN      CHINA               CHN   
281807  20161021     201610  2016        CHN      CHINA               CHN   
60624   20161206     201612  2016        CHN    CHINESE               CHN   
60625   20161206     201612  2016        CHN    CHINESE               CHN   
60633   20161206     201612  2016     CHNGOV      CHINA               CHN   

       Actor2Code  Actor2Name Actor2CountryCode  QuadClass  ...  \
41394         NaN         NaN               NaN          3  ...   
281807        NaN         NaN               NaN          1  ...   
60624         UZB       UZBEK               UZB          1  ...   
60625         UZB  UZBEKISTAN               UZB          1  ...   
60633         UZB  UZBEKISTAN               UZB          1  ...   

        Actor1Geo_Lat  Actor1Geo_Long  Actor2Geo_Type  Actor2Geo_CountryCode  \
41394        -10.0000             -55               0                    NaN   
281807        22.1558         113.557               0                    NaN   
60624         41.0000              64               1                     CH   
60625         41.0000              64               1                     CH   
60633         40.5000            47.5               1                     UZ   

       Actor2Geo_Lat  Actor2Geo_Long ActionGeo_Type  ActionGeo_CountryCode  \
41394            NaN             NaN              1                     BR   
281807           NaN             NaN              4                     MC   
60624           35.0             105              1                     CH   
60625           35.0             105              1                     UZ   
60633           41.0              64              1                     UZ   

       ActionGeo_Lat  ActionGeo_Long  
41394       -10.0000             -55  
281807       22.1558         113.557  
60624        35.0000             105  
60625        41.0000              64  
60633        41.0000              64  

[5 rows x 25 columns]

Mean of average tones for News Related to US (Actor1 is US):

In [67]:
dask_feat[dask_feat.Actor1CountryCode == 'USA'].AvgTone.mean().compute()

-1.8745312234584668

Mean of average tones for News Related to US (Actor2 is US):

In [68]:
dask_feat[dask_feat.Actor2CountryCode == 'USA'].AvgTone.mean().compute()

-1.9339521799661827

### News Related to Certain Topics

#### Topic 1: Donald Trump

In [80]:
mydf.select("SOURCEURL", mydf.SOURCEURL.like("% Trump %")).show(15)

+--------------------+------------------------+
|           SOURCEURL|SOURCEURL LIKE % Trump %|
+--------------------+------------------------+
|http://www.thesta...|                   false|
|http://asia.nikke...|                   false|
|http://asia.nikke...|                   false|
|http://asia.nikke...|                   false|
|http://www.thesta...|                   false|
|http://asia.nikke...|                   false|
|http://www.thesta...|                   false|
|http://www.messen...|                   false|
|http://www.messen...|                   false|
|http://www.messen...|                   false|
|http://krwg.org/p...|                   false|
|http://www.kltv.c...|                   false|
|http://www.michig...|                   false|
|http://missoulian...|                   false|
|http://www.intern...|                   false|
+--------------------+------------------------+
only showing top 15 rows



In [90]:
trump_news = mydf[mydf.SOURCEURL.like("%Trump%") == "true"]

In [91]:
trump_news.select("SOURCEURL").show(10)

+--------------------+
|           SOURCEURL|
+--------------------+
|http://www.4-trad...|
|http://m.chron.co...|
|http://www.gulf-t...|
|http://www.4-trad...|
|http://www.gulf-t...|
|http://www.gulf-t...|
|http://www.4-trad...|
|http://www.gulf-t...|
|http://www.onenew...|
|http://www.onenew...|
+--------------------+
only showing top 10 rows



In [ ]:
# trump_news.AvgTone.mean().compute()

In [ ]:
# display(dask_feat.groupBy('Actor1Geo_CountryCode')
#           .count()
#           .orderBy('count', ascending=False))

In [58]:
# spark.sql('select count(*) from dask_feat as count where (Actor1_CountryCode == 'CHN') and (Actor2_CountryCode == 'USA')')
# # spark.sql('select min(datetime) from  where (fnard == -1) and (fnok == -1) and (cark == -1) and (gnuck == -1)').collect()

SyntaxError: invalid syntax (<ipython-input-58-51fa351b3829>, line 1)

In [ ]:
import matplotlib.pyplot as plt

In [57]:
import matplotlib.pyplot as plt
##Visualizing most reviewed businesses through Barplot
most_extreme_tone \
.plot(kind='barh',stacked=False, figsize=[10,8], legend=False)
######
plt.title('Most Extreme News (Positive Tone)')
plt.xlabel('Avg Tone')
plt.ylabel('News ID')
plt.show()

In [ ]:
###Arranging cities with decreasing business counts
city_business_counts_dask = df[['city', 'business_id']].groupby(['city'])['business_id'].agg('count')
##Visualizing top 20 cites by businesses listed
city_business_counts_dask.nlargest(20).compute().\
plot(kind='barh', stacked=False, figsize=[10,10], colormap='hsv')

In [22]:
# mydf.write.save("s3://zandras3/502final/news.csv", format='csv', header=True)
# dask_df = dd.read_csv('s3://zandras3/502final/news.csv/*.csv')
# dask_df

# from pandas.io.common import EmptyDataError

# file = 's3://zandras3/502final/news.csv'

# def read_data(file):
#     try:
#         df = pd.read_csv(file, delim_whitespace=True)
#         print("yes")
#     except EmptyDataError:
#         df = pd.DataFrame()
#         print("no")

# #     return df

In [62]:
import sys
import os,datetime, re
from pyspark.sql import Row